# <font color='Crimson'><b>VARIABLE SELECTION</b></font>

In [ ]:
#Import packages:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler 
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import make_scorer
#from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import statistics
from sklearn.metrics import class_likelihood_ratios
from IPython.display import clear_output

#### <font color='indianred'><b>FEATURE IMPORTANCE</b></font>

In [ ]:
#Import imputed and standardized training set:
training_std = pd.read_csv('05_Training_standard.csv',sep=',',index_col=0)
training_std.head()

In [ ]:
#Import imputed and standardized test set:
test_std = pd.read_csv('05_Test_standard.csv',sep=',',index_col=0)
test_std.head()

In [ ]:
#Shuffle the two sets:
training_std = shuffle(training_std,random_state=100)
test_std = shuffle(test_std,random_state=100)

In [ ]:
#Split features from outcome in both sets:
X_training = training_std.iloc[:,2:training_std.shape[1]].values
y_training = training_std.iloc[:,0].values
y_training = np.double(y_training)
X_test = test_std.iloc[:,2:test_std.shape[1]].values
y_test = test_std.iloc[:,0].values
y_test = np.double(y_test)

In [ ]:
#Define useful functions for classification task:
#TSS score
def my_tss_score(Y_training, probability_prediction,threshold):
    
    Y_predicted = probability_prediction > threshold
    res = metrics_classification(Y_training > 0, Y_predicted, print_skills=False)    

    return res['tss']

#Calssification metrics
def metrics_classification(y_real, y_pred, print_skills=True):

    cm, far, pod, acc, hss, tss, fnfp, csi, tpr, tnr = classification_skills(y_real, y_pred)

    if print_skills:
        print ('confusion matrix')
        print (cm)
        print ('false alarm ratio       \t', far)
        print ('probability of detection\t', pod)
        print ('accuracy                \t', acc)
        print ('hss                     \t', hss)
        print ('tss                     \t', tss)
        print ('balance                 \t', fnfp)
        print ('csi                 \t', csi)
        print ('tpr                 \t', tpr)
        print ('tnr                 \t', tnr)

    balance_label = float(sum(y_real)) / y_real.shape[0]

    return {
        "cm": cm,
        "far": far,
        "pod": pod,
        "acc": acc,
        "hss": hss,
        "tss": tss,
        "fnfp": fnfp,
        "balance label": balance_label,
        "csi": csi,
        "tpr": tpr,
        "tnr": tnr}

def classification_skills(y_real, y_pred):

    cm = confusion_matrix(y_real, y_pred)

    if cm.shape[0] == 1 and sum(y_real) == 0:
        a = 0.
        d = float(cm[0, 0])
        b = 0.
        c = 0.
    elif cm.shape[0] == 1 and sum(y_real) == y_real.shape[0]:
        a = float(cm[0, 0])
        d = 0.
        b = 0.
        c = 0.
    elif cm.shape[0] == 2:
        a = float(cm[1, 1])
        d = float(cm[0, 0])
        b = float(cm[0, 1])
        c = float(cm[1, 0])
    TP = a
    TN = d
    FP = b
    FN = c

    if (TP + FP + FN + TN) == 0.:
        if (TP + TN) == 0.:
            acc = 0.  # float('NaN')
        else:
            acc = -100  # float('Inf')
    else:
        acc = (TP + TN) / (TP + FP + FN + TN)
        

    if TP + FN == 0.:
        if TP == 0.:
            tss_aux1 = 0.  # float('NaN')
        else:
            tss_aux1 = -100  # float('Inf')
    else:
        tss_aux1 = (TP / (TP + FN))

    if (FP + TN) == 0.:
        if FP == 0.:
            tss_aux2 = 0.  # float('NaN')
        else:
            tss_aux2 = -100  # float('Inf')
    else:
        tss_aux2 = (FP / (FP + TN))

    tss = tss_aux1 - tss_aux2

    if ((TP + FN) * (FN + TN) + (TP + FP) * (FP + TN)) == 0.:
        if (TP * TN - FN * FP) == 0:
            hss = 0.  # float('NaN')
        else:
            hss = -100  # float('Inf')
    else:
        hss = 2 * (TP * TN - FN * FP) / ((TP + FN) *
                                         (FN + TN) + (TP + FP) * (FP + TN))

    if FP == 0.:
        if FN == 0.:
            fnfp = 0.  # float('NaN')
        else:
            fnfp = -100  # float('Inf')
    else:
        fnfp = FN / FP

    if (TP + FN) == 0.:
        if TP == 0.:
            pod = 0  # float('NaN')
        else:
            pod = -100  # float('Inf')
    else:
        pod = TP / (TP + FN)


    if (TP + FP) == 0.:
        if FP == 0.:
            far = 0.  # float('NaN')
        else:
            far = -100  # float('Inf')
    else:
        far = FP / (TP + FP)

    #acc = (a + d) / (a + b + c + d)
    tpr = tss_aux1  # a / (a + b)
    tnr = 1-tss_aux2  # d / (d + c)
    #wtpr = a / (a + b) * (a + c) / (a + b + c + d) + d / (c + d) * (b + d) / (a + b + c + d)
    #pacc = a / (a + c)
    #nacc = d / (b + d)
    #wacc = a / (a + c) * (a + c) / (a + b + c + d) + d / (b + d) * (b + d) / (a + b + c + d)

    # if the cm has a row or a column equal to 0, we have bad tss
    if TP+FN == 0 or TN+FP == 0 or TP+FP == 0 or TN+FN == 0:
        tss = 0
    if TP+FP+FN==0:
        csi = 0
    else:
        csi = TP/(TP+FP+FN)

    return cm.tolist(), far, pod, acc, hss, tss, fnfp, csi, tpr, tnr

In [ ]:
#Define useful fuctions for cross-validation:
def build_grid(p):
    #Parameters initialization:
    model_name = None if p['model_name'] == None else p['model_name']
    GRID = None 

    if model_name == "RF":
        GRID = {'n_estimators': p['n_estimators'],
                'max_features': p['max_features'],
                'max_depth': p['max_depth'],
                'criterion': p['criterion']
               }   

    #other models can be added here

    return GRID


def initiate_p(p):
    #Ranges for hyperparameter search:
    if p['model_name'] == 'RF':
        p['estimator'] = RandomForestClassifier(random_state=100)
        p['n_estimators'] = [100,200,300,400,500]
        p['max_features'] = [None, 'sqrt', 'log2'] 
        p['max_depth'] = [4,5,6,7,8]
        p['criterion'] = ['gini','entropy']

    #other models can be added here

    #All:
    p['cv'] = 10

    return p


def GridSearch_(X,y,p):
    #Parameters initialization:
    estimator = None if p['estimator'] == None else p['estimator']
    cv = 10 if p['cv'] == None else p['cv']
    grid = None if p['grid'] == None else p['grid']
    tau = 0.5 if p['threshold'] == None else p['threshold']
    
    if(estimator == None or grid == None):
        return None

    #1st step: select the best hyperparameters
    CV = GridSearchCV(estimator     = estimator,
                        param_grid  = grid,
                        scoring     =  make_scorer(my_tss_score,threshold=tau,needs_proba=True), #TSS score
                        #refit       = 'roc_auc',
                        cv          = cv,
                        verbose     = 0,
                        n_jobs      = 20,
                        return_train_score=True)
    CV_H = CV.fit(X,y)
    
    return CV_H

In [ ]:
#Define the function that returns the ranking of features, from the most important to the less one:

#Default scoring function:
scoring = make_scorer(my_tss_score,threshold=0.10,needs_proba=True)

#Feature ranking:
def feature_importance(model,X,y,df,n_repeats=30, random_state=100, scoring=scoring):
    r = permutation_importance(model, X, y, n_repeats=30, random_state=100, scoring=scoring)
    feature_rel = []
    mean_value_features=[]
    std_value_features=[]
    for i in r.importances_mean.argsort()[::-1]:
        if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            print(f"{df.columns[i]:<8}"
               f"{r.importances_mean[i]:.3f}"
               f" +/- {r.importances_std[i]:.3f}")
            feature_rel.append(df.columns[i])
            mean_value_features.append(r.importances_mean[i])
            std_value_features.append(r.importances_std[i])
    return feature_rel,mean_value_features,std_value_features

In [ ]:
#Compute feature importance for each of the 3*10 folds trained before:
#On validation set:
dataset = 'validation' 
#Number of shuffles:
R = 3
#Number of folds:
K = 10

#Thresholds evaluated:
thresholds = np.linspace(0.0,0.5,11)
#Classifier trained:
classifiers = ['RF'] #LogisticRegL1 #LogisticRegL2

#Shuffle data in order to test stability of the model:
#In order to guarantee reproducibility of results, the random states are always the numbers 0,1,2.
for r in range(0,R):
    X,y = shuffle(X_training,y_training,random_state = r) 
    for k in range(0,K):
        for cl in classifiers:
            p = {}
            p['model_name'] = cl
            p = initiate_p(p)  
            #Read from file:
            #insert your path here:
            nome_file = "<insert your path>/R"+ str(r) + "_K" + str(k) + '_' + p['model_name'] + '.npy' #"K-Folds/R"
            metrics = np.load(nome_file, allow_pickle = True).item()
            
            index = ''
            if dataset == 'validation':
                index = 'validation_index'
            elif dataset == 'train':
                index = 'train_index'
                
            index = metrics[index]
            X_kfold = X[index]
            y_kfold = y[index]
            
            best_thresholds = {}
            best_threshold = -1
            best_hyperparameters = {}
            
            metrics_cl = metrics[cl]
            
            for th in thresholds:
                performance = metrics_cl[th]
                
                if dataset == 'validation':
                    sensitivity = performance['recall_ts']
                    specificity = performance['specificity_ts']
                    tss = performance['tss_ts']
                elif dataset == 'train':
                    sensitivity = performance['recall_tr']
                    specificity = performance['specificity_tr']
                    tss = performance['tss_tr']
                
                #Relationship of interest:
                if sensitivity >= specificity:
                    if round(specificity,2) >= 0.55:
                        if round(sensitivity,2) >= 0.65: 
                            best_thresholds[th] = tss
                
            if len(best_thresholds) == 0:
                print('No threshold satisfies the relationship!')
            elif len(best_thresholds) == 1:
                for key, value in best_thresholds.items():
                    best_threshold = key
            else:
                best_threshold = max(best_thresholds, key=best_thresholds.get)

            if best_threshold == -1:
                fi = {}
                fi['Features_names'] = []
                fi['Mean_values'] = []
                fi['Std_values'] = []
                #nome_file = "<insert your path>/R"+ str(r) + "_K" + str(k) + '_' + p['model_name'] + '_FeaturesImportance_'+dataset + '.npy' 
                #np.save(nome_file, fi) 
                continue
                
            print("Best threshold:",best_threshold)
            
            performance = metrics_cl[best_threshold]
            
            best_hyperparameters = performance['best_hyper']
            
            print("Best set of hyperparameters:",best_hyperparameters)

            for key, value in best_hyperparameters.items():
                p[key] = value

            scoring = make_scorer(my_tss_score,threshold=best_threshold,needs_proba=True)

            if p['model_name'] == 'LogisticRegL1':
                fit = LogisticRegressionCV(cv = 1,random_state = 100,penalty='l1',solver='liblinear',scoring=scoring).fit(X_kfold, y_kfold) 

            elif p['model_name'] == 'LogisticRegL2':
                fit = LogisticRegressionCV(cv = 1,random_state = 100,penalty='l2',scoring=scoring).fit(X_kfold, y_kfold) 

            elif p['model_name'] == 'RF':
                max_depth = p['max_depth']
                n_estimators = p['n_estimators']
                criterion = p['criterion']
                max_features = p['max_features']
                fit = RandomForestClassifier(max_depth=max_depth,n_estimators=n_estimators,criterion=criterion,
                                             max_features=max_features,random_state=100).fit(X_kfold, y_kfold) 
                
            #other models can be added here
                
                
            feature_rel,mean_value,std_value = feature_importance(fit,X_kfold,y_kfold,
                                                                  train_std.iloc[:,2:training_std.shape[1]],
                                                                  n_repeats=30, random_state=100, 
                                                                  scoring=scoring)  
            
            fi = {}
            fi['Features_names'] = feature_rel
            fi['Mean_values'] = mean_value
            fi['Std_values'] = std_value
            #df_fi = pd.DataFrame(fi)
            #Save on file:
            #nome_file = "<insert your path>/R"+ str(r) + "_K" + str(k) + '_' + p['model_name'] + '_FeaturesImportance_'+dataset + '.npy'
            #np.save(nome_file, fi)                   

In [ ]:
#Evaluate all the ranks and the best position reached by each feature:
all_features = ['SESSO','30gg','ETA','BASOFILI','EOSINOFILI','LINFOCITI','MONOCITI','NEUTROFILI','EMATOCRITO','EMOGLOBINA',
                'GLOBULI_B','GLOBULI_R','PIASTRINE','APTT','INR','TEMPO_PROTROMB','ACIDO_URICO','ALP','ALT','AST',
                'BILIRUBINA_TOT','CREATININA','GGT','LAD','UREA','GLUCOSIO','ALBUMINA','PCR','PROTEINE_TOT']

#Best ranks:
best_rank = {} #Key:Feature, Value: Best rank
#All ranks:
all_ranks = {} #Key: Feature, Value:Vocabulary -> 1:n, 2:n, ...
ranks = {}
for i in range(1,len(all_features)+1):
    ranks[i] = 0

#Initialization of the two dictionaries' keys:
for f in all_features:
    best_rank[f] = 100
    
    all_ranks[f] = ranks

#Update dictionaries:
for r in range(0,R):
    for k in range(0,K):
        for cl in classifiers:
            model_name = cl   
            #Read from file:
            #insert your path here:
            nome_file = "<insert your path>/R"+ str(r) + "_K" + str(k) + '_' + p['model_name'] + '_FeaturesImportance_'+dataset + '.npy' 
            fi = np.load(nome_file, allow_pickle = True).item()
            Features_names = fi['Features_names']
            
            rank = 1
            for name in Features_names:
                if best_rank[name] > rank:
                    best_rank[name] = rank
                                
                if rank <= len(all_features):
                    ranks = all_ranks[name].copy()
                    all_ranks[name] = ranks
                    ranks[rank] += 1
                    
                rank += 1
                
print(best_rank)
print(all_ranks)
#Save on file:
#nome_file = "<insert your path>/best_rank.npy"
#np.save(nome_file, best_rank)   
#nome_file = "<insert your path>/all_ranks.npy"
#np.save(nome_file, all_ranks)

In [ ]:
#insert your path in the following 2 lines:
#best_rank = np.load('<insert your path>/best_rank.npy', allow_pickle = True).item()
all_ranks = np.load('<insert your path>/all_ranks.npy', allow_pickle = True).item()

In [ ]:
#Change column names for Figure S3:
all_features = ['SESSO','30gg','ETA','BASOFILI','EOSINOFILI','LINFOCITI','MONOCITI','NEUTROFILI','EMATOCRITO','EMOGLOBINA',
                'GLOBULI_B','GLOBULI_R','PIASTRINE','APTT','INR','TEMPO_PROTROMB','ACIDO_URICO','ALP','ALT','AST',
                'BILIRUBINA_TOT','CREATININA','GGT','LAD','UREA','GLUCOSIO','ALBUMINA','PCR','PROTEINE_TOT']
new_keys = ["Sex","Candida colonization","Age","Basophil cells count","Eosinophil cells count",
            "Lymphocyte cells count","Monocyte cells count","Neutrophil cells count","Hematocrit","Hemoglobin",
            "White cells count","Red cells count", "Platelets count","aPTT","INR","Prothrombin time","Uric acid",
            "ALP","ALT","AST","Total bilirubin","Creatinine","GGT","LDH","Urea","Glucose","Albumin","CRP","Total proteins"]
    
def change_dict_key(d, old_key, new_key, default_value=None):
    d[new_key] = d.pop(old_key, default_value)
    
for i in range(0,29):
    change_dict_key(all_ranks, all_features[i], new_keys[i])

In [ ]:
#Visulize histograms for all ranks (Figure S3):
#Add to the dictionary 'all_ranks' the number of times each feature is not important 
#(therefore is not displaied by feature importance). This number has key 100
#all_ranks_nan = all_ranks.copy() 
#for key,val in all_ranks_nan.items():
    #val.update({100: 23 - sum(list(val.values()))}) # 23 is the number of non-empty files
    
#Histograms (Figure S3):
for key,value in all_ranks.items():
    pos = np.arange(len(value.keys()))
    width = 0.5     

    ax = plt.axes()
    ax.set_box_aspect(0.5)
    ax.set_ylim(0.0,27.0)
    yticks = [0,5,10,15,20,23]
    ax.set_yticklabels(yticks,fontdict={'fontsize' : 6.5})
    ax.set_xlim(0.0,30)
    ax.set_xticks(pos + width*2)
    ax.set_xticklabels(list(value.keys()),fontdict={'fontsize' : 6.5})
    ax.set_xlabel('Position')
    ax.set_ylabel('Frequency')    
    ax.set_title(key)

    plt.bar(value.keys(),value.values(), width, color='#80b1d3')
    
    #nome_figura = '<insert your path>/histogram_rank_' + key + '.pdf'
    #plt.savefig(nome_figura,dpi = 300,bbox_inches='tight')
    
    plt.show()

#### <font color='indianred'><b>SUBSET WITH BDG AND PCT NON-MISSING</b></font>

In [ ]:
#Import dataset:
df_complete1 = pd.read_csv('02_1_Dataset.csv', sep=',',index_col=0)
df_complete1.head()

In [ ]:
#Select BDG and PCT columns:
df_complete1_BDG_PCT = df_complete1.iloc[:,[40,42]]
print('The number of episodes with BDG and PCT non-missing is:', 
      df_complete1_BDG_PCT[~(df_complete1_BDG_PCT['PROCALCITONINA'].isna() | df_complete1_BDG_PCT['B_D_GLUCANO'].isna())].shape[0])

In [ ]:
#Import training features:
X_training = pd.read_csv('03_X_training.csv',sep=',',index_col=0)
X_training.head()

In [ ]:
#Import training outcome:
y_training = pd.read_csv('03_y_training.csv',sep=',',index_col=0)
y_training.head()

In [ ]:
#Import test features:
X_test = pd.read_csv('03_X_test.csv',sep=',',index_col=0)
X_test.head()

In [ ]:
#Import test outcome:
y_test = pd.read_csv('03_y_test.csv',sep=',',index_col=0)
y_test.head()

In [ ]:
#Merge training features and outcome in training set:
training = X_training.drop(X_training.columns[[0,3,4,5,8]], axis=1)
training['CANDIDEMIA'] = y_training
training.head()

In [ ]:
#Merge the training set and BDG and PCT columns:
df_complete1_training = df_complete1_BDG_PCT.merge(training, how='right',right_index=True,left_index=True)
df_complete1_training.head()

In [ ]:
#Delete episodes with missing BDG or PCT:
training_BDG_PCT = df_complete1_training[~(df_complete1_training['PROCALCITONINA'].isna() | df_complete1_training['B_D_GLUCANO'].isna())]
print('Number of samples in the new training set:',training_BDG_PCT.shape[0])

In [ ]:
print('Are missing values left in the training set?',training_BDG_PCT.isnull().values.any())

In [ ]:
#Imputation
#Definition of the imputer:
imputer = IterativeImputer(KNeighborsRegressor(n_neighbors=5),
                           sample_posterior=False, 
                           max_iter=100,
                           tol=0.05,
                           n_nearest_features=None,
                           initial_strategy='most_frequent',
                           imputation_order='random',
                           random_state=100)

In [ ]:
#Fit of the imputer on training set and imputation:
training_BDG_PCT_imputed = imputer.fit_transform(training_BDG_PCT)
training_BDG_PCT_imputed = pd.DataFrame(training_BDG_PCT_imputed)

In [ ]:
print('Are missing values left in the training set?',training_BDG_PCT_imputed.isnull().values.any())

In [ ]:
#Work on imputed training set:
training_BDG_PCT_imputed.columns = training_BDG_PCT.columns
training_BDG_PCT_imputed['30gg'] = training_BDG_PCT_imputed['30gg'] > 0.5
training_BDG_PCT_imputed['30gg'] = training_BDG_PCT_imputed['30gg']*1.
training_BDG_PCT_imputed.head()

In [ ]:
#Standardization
#Change the order of the columns of the imputed training set:
training_BDG_PCT_imputed = training_BDG_PCT_imputed.reindex(columns=['CANDIDEMIA','MISTA','SESSO','30gg','B_D_GLUCANO','PROCALCITONINA','ETA','BASOFILI',
                                                                     'EOSINOFILI','LINFOCITI','MONOCITI','NEUTROFILI','EMATOCRITO','EMOGLOBINA','GLOBULI_B',
                                                                     'GLOBULI_R','PIASTRINE','APTT','INR','TEMPO_PROTROMB','ACIDO_URICO','ALP','ALT','AST',
                                                                     'BILIRUBINA_TOT','CREATININA','GGT','LAD','UREA','GLUCOSIO','ALBUMINA','PCR','PROTEINE_TOT'])
training_BDG_PCT_imputed.head()

In [ ]:
#Define the scaler:
scaler = StandardScaler()

In [ ]:
#Fit of the scaler on the continuous features of the training set and scale:
X_training_BDG_PCT_std = pd.DataFrame(scaler.fit_transform(training_BDG_PCT_imputed.iloc[:,4:training_BDG_PCT_imputed.shape[1]]))

In [ ]:
#Add categorical features to the one just scaled:
training_BDG_PCT_std = pd.DataFrame(np.concatenate((training_BDG_PCT_imputed.iloc[:,0:4], X_training_BDG_PCT_std), axis=1))
#Change column names:
training_BDG_PCT_std.columns = training_BDG_PCT_imputed.columns
training_BDG_PCT_std.head()

In [ ]:
#Merge test features and outcome in training set:
test = X_test.drop(X_test.columns[[0,3,4,5,8]], axis=1)
test['CANDIDEMIA'] = y_test
test.head()

In [ ]:
#Merge the test set and BDG and PCT columns:
df_complete1_test = df_complete1_BDG_PCT.merge(test, how='right',right_index=True,left_index=True)
df_complete1_test.head()

In [ ]:
#Delete episodes with missing BDG or PCT:
test_BDG_PCT = df_complete1_test[~(df_complete1_test['PROCALCITONINA'].isna() | df_complete1_test['B_D_GLUCANO'].isna())]
print('Number of samples in the new test set:',test_BDG_PCT.shape[0])

In [ ]:
print('Are missing values left in the test set?',test_BDG_PCT.isnull().values.any())

In [ ]:
#Imputation
#Imputation of the test set:
test_BDG_PCT_imputed = imputer.transform(test_BDG_PCT)
test_BDG_PCT_imputed = pd.DataFrame(test_BDG_PCT_imputed)

In [ ]:
print('Are missing values left in the test set?',test_BDG_PCT_imputed.isnull().values.any())

In [ ]:
#Work on imputed test set:
test_BDG_PCT_imputed.columns = test_BDG_PCT.columns
test_BDG_PCT_imputed['30gg'] = test_BDG_PCT_imputed['30gg'] > 0.5
test_BDG_PCT_imputed['30gg'] = test_BDG_PCT_imputed['30gg']*1.
test_BDG_PCT_imputed.head()

In [ ]:
#Standardization
#Change the order of the columns of the imputed test set:
test_BDG_PCT_imputed = test_BDG_PCT_imputed.reindex(columns=['CANDIDEMIA','MISTA','SESSO','30gg','B_D_GLUCANO','PROCALCITONINA','ETA','BASOFILI','EOSINOFILI',
                                                             'LINFOCITI','MONOCITI','NEUTROFILI','EMATOCRITO','EMOGLOBINA','GLOBULI_B','GLOBULI_R','PIASTRINE',
                                                             'APTT','INR','TEMPO_PROTROMB','ACIDO_URICO','ALP','ALT','AST','BILIRUBINA_TOT','CREATININA','GGT',
                                                             'LAD','UREA','GLUCOSIO','ALBUMINA','PCR','PROTEINE_TOT'])
test_BDG_PCT_imputed.head()

In [ ]:
#Scale the continuous features of the test set:
X_test_BDG_PCT_std = pd.DataFrame(scaler.transform(test_BDG_PCT_imputed.iloc[:,4:test_BDG_PCT_imputed.shape[1]]))

In [ ]:
#Add categorical features to the one just scaled:
test_BDG_PCT_std = pd.DataFrame(np.concatenate((test_BDG_PCT_imputed.iloc[:,0:4], X_test_BDG_PCT_std), axis=1))
#Change column names:
test_BDG_PCT_std.columns = test_BDG_PCT_imputed.columns
test_BDG_PCT_std.head()

In [ ]:
#New percentage of candidemias:
print('Number of candidemias in the new training set:', pd.value_counts(training_BDG_PCT_std['CANDIDEMIA'])[1], 
      '(',round(pd.value_counts(training_BDG_PCT_std['CANDIDEMIA'], normalize=True)[1]*100,2),'%) of',
      training_BDG_PCT_std['CANDIDEMIA'].shape[0], 'samples')
print('--------------------------------------------------------------')
print('Number of candidemias in the new test set:', pd.value_counts(test_BDG_PCT_std['CANDIDEMIA'])[1], 
      '(',round(pd.value_counts(test_BDG_PCT_std['CANDIDEMIA'], normalize=True)[1]*100,2),'%) of',
      test_BDG_PCT_std['CANDIDEMIA'].shape[0], 'samples')

In [ ]:
#Shuffle the two sets:
training_BDG_PCT_std = shuffle(training_BDG_PCT_std,random_state=100)
test_BDG_PCT_std = shuffle(test_BDG_PCT_std,random_state=100)

In [ ]:
#Save on csv file:
training_BDG_PCT_std.to_csv('07_Training_BDG_PCT_standard.csv',sep=',') 
test_BDG_PCT_std.to_csv('07_Test_BDG_PCT_standard.csv',sep=',') 

#### <font color='indianred'><b>WITH FEATURES SELECTED BASED ON ALL THEIR RANKS</b></font>

In [ ]:
#Import imputed and standardized training set:
training_BDG_PCT_std = pd.read_csv('07_Training_BDG_PCT_standard.csv',sep=',',index_col=0) 
training_BDG_PCT_std.head()

In [ ]:
#Import imputed and standardized test set:
test_BDG_PCT_std= pd.read_csv('07_Test_BDG_PCT_standard.csv',sep=',',index_col=0) 
test_BDG_PCT_std.head()

In [ ]:
#insert your path here:
all_ranks = np.load('<insert your path>/all_ranks.npy', allow_pickle = True).item()

In [ ]:
#Count the number of times that each feature is in one of the first N positions
#Choice for N: 
N = 18 #chosen looking at the previous histograms

#Delete from the dictionary all_ranks values with key > N:
all_ranks_N = all_ranks.copy()
for key,val in all_ranks_N.items():
    for i in range(N+1,30):
        del all_ranks_N[key][i] 

n_importance_N = {} #Key:feature, Value:n
for key,val in all_ranks_N.items():
    n_importance_N[key] = sum(list(val.values()))

subsets = {} # Key:i, Value:[features]
for i in range(30,-1,-1):
    subset = {}
    for feature,n in n_importance_N.items():
        if n >= i:
            subset[feature] = n
    features = list(subset.keys())
    subsets[i] = features   

#Delete duplicates from the previous dictionary:
temp = []
subsets_unique = dict()

for key, val in subsets.items():
    if val not in temp:
        temp.append(val)
        subsets_unique[key] = val
        
#print(subsets_unique)

for key, val in subsets_unique.items():        
    #Add the feature subset to Candidemia, BDG and PCT in both sets:
    sub2_BDG_PCT_y_training = training_BDG_PCT_std[['CANDIDEMIA','B_D_GLUCANO','PROCALCITONINA']+val]
    sub2_BDG_PCT_y_test = test_BDG_PCT_std[['CANDIDEMIA','B_D_GLUCANO','PROCALCITONINA']+val]

    #Splitting features and outcome:
    X_training = sub2_BDG_PCT_y_training.iloc[:,1:sub2_BDG_PCT_y_training.shape[1]].values
    y_training = sub2_BDG_PCT_y_training.iloc[:,0].values
    y_training = np.double(y_training)
    X_test = sub2_BDG_PCT_y_test.iloc[:,1:sub2_BDG_PCT_y_test.shape[1]].values
    y_test = sub2_BDG_PCT_y_test.iloc[:,0].values
    y_test = np.double(y_test)

    #Range of threshold to evaluate:
    thresholds = np.linspace(0.0,0.5,11) 

    #Classifiers to train:
    classifiers = ['RF'] #LogisticRegL1 #LogisticRegL2
    metrics_cl = {}

    for cl in classifiers:
        p = {}
        p['model_name'] = cl
        p = initiate_p(p)
        print('Model name:', p['model_name'])

        metrics_th = {}

        for th in tqdm(thresholds):
            p['threshold'] = th
            print('Threshold:', p['threshold'])

            performance = {}
            performance['features'] = sub2_BDG_PCT_y_training.columns
            #---------------------------------------------
            #Hyperparameters search and model fit on training set:    
            if p['model_name'] == 'LogisticRegL1':
                score = make_scorer(my_tss_score,threshold=th,needs_proba=True)
                fit = LogisticRegressionCV(cv = 10,random_state = 100,penalty='l1',solver='liblinear',scoring=score).fit(X_training, y_training) 

            elif p['model_name'] == 'LogisticRegL2':
                score = make_scorer(my_tss_score,threshold=th,needs_proba=True)
                fit = LogisticRegressionCV(cv = 10,random_state = 100,penalty='l2',scoring=score).fit(X_training, y_training) 

            elif p['model_name'] == 'RF':
                p['grid'] = build_grid(p)
                p['GS_RF'] = GridSearch_(X_training, y_training, p)
                max_depth = p['GS_RF'].best_params_['max_depth']
                n_estimators = p['GS_RF'].best_params_['n_estimators']
                criterion = p['GS_RF'].best_params_['criterion']
                max_features = p['GS_RF'].best_params_['max_features']
                print(p['GS_RF'].best_params_)
                performance['best_hyper'] = p['GS_RF'].best_params_
                fit = RandomForestClassifier(max_depth=max_depth,n_estimators=n_estimators,
                                             criterion=criterion,max_features=max_features,random_state=100).fit(X_training, y_training) 

            #other models can be added here
            
            #---------------------------------------------
            #Predict on training set:
            PRED_prob_tr = fit.predict_proba(X_training)
            PRED_tr_yes = PRED_prob_tr[:,1] 

            PRED_tr_bin = PRED_tr_yes > th
            PRED_tr_bin = PRED_tr_bin*1.

            performance['ytr'] = y_training
            performance['PRED_prob_tr'] = PRED_prob_tr

            #Compute evaluation metrics on training set:
            print(confusion_matrix(y_training,PRED_tr_bin))
            tn_tr, fp_tr, fn_tr, tp_tr = confusion_matrix(y_training, PRED_tr_bin).ravel()
            spec_tr = tn_tr / (tn_tr+fp_tr)
            f1_tr = f1_score(y_training, PRED_tr_bin,average = 'weighted')
            acc_tr = accuracy_score(y_training, PRED_tr_bin)
            prec_tr = precision_score(y_training, PRED_tr_bin,average = 'weighted')
            recall_tr = recall_score(y_training, PRED_tr_bin) 
            npv_tr = tn_tr / (tn_tr+fn_tr)

            performance['tss_tr'] = recall_tr+spec_tr-1
            performance['f1score_tr'] = f1_tr
            performance['accuracy_tr'] = acc_tr
            performance['precision_tr'] = prec_tr
            performance['recall_tr'] = recall_tr
            performance['specificity_tr'] = spec_tr
            performance['npv_tr'] = npv_tr 

            #---------------------------------------------
            #Predict on test set:
            PRED_prob_ts = fit.predict_proba(X_test)
            PRED_ts_yes = PRED_prob_ts[:,1]

            PRED_ts_bin = PRED_ts_yes>th
            PRED_ts_bin = PRED_ts_bin*1.

            performance['yts'] = y_test
            performance['PRED_prob_ts'] = PRED_prob_ts

            #Compute evaluation metrics on test set:
            print(confusion_matrix(y_test,PRED_ts_bin))
            tn_ts, fp_ts, fn_ts, tp_ts = confusion_matrix(y_test,PRED_ts_bin).ravel()
            spec_ts = tn_ts / (tn_ts+fp_ts)
            f1_ts = f1_score(y_test, PRED_ts_bin,average = 'weighted')
            acc_ts = accuracy_score(y_test, PRED_ts_bin)
            prec_ts = precision_score(y_test, PRED_ts_bin,average = 'weighted')
            recall_ts = recall_score(y_test, PRED_ts_bin)  
            npv_ts = tn_ts / (tn_ts+fn_ts)

            performance['tss_ts'] = recall_ts+spec_ts-1
            performance['f1score_ts'] = f1_ts
            performance['accuracy_ts'] = acc_ts
            performance['precision_ts'] = prec_ts
            performance['recall_ts'] = recall_ts
            performance['specificity_ts'] = spec_ts
            performance['npv_ts'] = npv_ts 
            #---------------------------------------------
            metrics_th[th] = performance
        #---------------------------------------------
        #Performances at each treshold:
        metrics_cl[cl] = metrics_th
        #Save on file:
        #nome_file = '<insert your path>/Cutoff' + str(N) + '_' + cl + '_Performances_rank'+ str(key)+'.npy' 
        #np.save(nome_file, metrics_th)
    #Save on file:
    #nome_file = '<insert your path>/Complete_Performances_Cutoff_' + str(N)+'.npy'
    #np.save(nome_file, metrics_cl)

In [ ]:
#insert your path here:
all_ranks = np.load('<insert your path>/all_ranks.npy', allow_pickle = True).item()

In [ ]:
#Performances visualization (TSS, accuracy, F1-score, precision, NPV) on training set (Figure S4):
N = [30,23,22,21,20,19,18,16,15,13,12,10,9,8,6,0] #key of subset_unique
thresholds = np.linspace(0.0,0.5,11)

for th in thresholds:
    n = [] #number of features plus BDG and PCT
    tss = []
    f1 = []
    acc = []
    prec = []
    se = []
    sp = []
    npv = []
    #lr_p = []
    #lr_n = []
    
    for i in N:
        #insert your path here:
        nome_file = '<insert your path>/Cutoff18_RF_Performances_rank' + str(i) +'.npy'
        file = np.load(nome_file, allow_pickle = True).item()
        metrics = file[th]
        
        n.append(len(metrics['features'])-3)
        tss.append(metrics['tss_tr'])
        f1.append(metrics['f1score_tr'])
        acc.append(metrics['accuracy_tr'])
        prec.append(metrics['precision_tr'])
        npv.append(metrics['npv_tr'])
        se.append(metrics['recall_tr'])
        sp.append(metrics['specificity_tr'])
        
        #Likelihood Ratios:
        #pos_LR_tr, neg_LR_tr = class_likelihood_ratios(metrics['ytr'],PRED_tr_bin)
        #lr_p.append(pos_LR_tr)
        #lr_n.append(neg_LR_tr)
        
    #performances to visualize:
    y = np.array([tss,f1,acc,prec,npv]).transpose() #,lr_p,lr_n
    plt.xlabel("N")
    plt.ylabel("Model performance")
    plt.ylim(-0.1, 1.1)
    #plt.title(str(th))

    legend = ['TSS', 'F1-score', 'Accuracy', 'Precision','NPV'] #, "LR+","LR-"
    colors = ['#fdb462','#b3de69','#bebada','#fb8072','#80b1d3'] #,'#8dd3c7','#ffffb3'

    for i in range(len(y[0])):
        plt.plot(n,[pt[i] for pt in y],label = legend[i], color = colors[i])
    plt.legend(loc=(0.82,0.22),fontsize='x-small')
    plt.xticks(n,size=6.5)
    plt.yticks(size=6.5)
    #nome_figura = '<insert your path>/Model_tss_f1_acc_prec_npv_trend_' + str(th) +'.png'
    #plt.savefig(nome_figura,dpi=300)
    plt.show()

In [ ]:
#Performances visualization (recall, SP) on training set (Figure S4):
N = [30,23,22,21,20,19,18,16,15,13,12,10,9,8,6,0] #key of subset_unique
thresholds = np.linspace(0.0,0.5,11)

for th in thresholds:
    n = [] #number of features plus BDG and PCT
    tss = []
    f1 = []
    acc = []
    prec = []
    se = []
    sp = []
    
    for i in N:
        #insert your pat here:
        nome_file = '<insert your path>/Cutoff18_RF_Performances_rank' + str(i) +'.npy'
        file = np.load(nome_file, allow_pickle = True).item()
        metrics = file[th]
        
        n.append(len(metrics['features'])-3)
        tss.append(metrics['tss_tr'])
        f1.append(metrics['f1score_tr'])
        acc.append(metrics['accuracy_tr'])
        prec.append(metrics['precision_tr'])
        npv.append(metrics['npv_tr'])
        se.append(metrics['recall_tr'])
        sp.append(metrics['specificity_tr'])
        
        #Likelihood Ratios:
        #pos_LR_tr, neg_LR_tr = class_likelihood_ratios(metrics['ytr'],PRED_tr_bin)
        #lr_p.append(pos_LR_tr)
        #lr_n.append(neg_LR_tr)
        
    #performances to visualize:
    y = np.array([se,sp]).transpose() 
    plt.xlabel("N")
    plt.ylabel("Model performance")
    plt.ylim(-0.1, 1.1)
    #plt.title(str(th))

    legend = ['Recall', 'Specificity']
    colors = ['#8dd3c7','#fff033']

    for i in range(len(y[0])):
        plt.plot(n,[pt[i] for pt in y],label = legend[i], color = colors[i])
    plt.legend(loc=(0.82,0.22),fontsize='x-small')
    plt.xticks(n,size=6.5)
    plt.yticks(size=6.5)
    plt.axhline(y=0.6, color='black', linestyle='-.',linewidth=0.7)
    #nome_figura = '<insert your path>/Model_se_sp_trend_' + str(th) +'.png'
    #plt.savefig(nome_figura,dpi=300)
    plt.show()

In [ ]:
#Zoom in threshold between 0.15 and 0.2:
N = 18 #chosen looking at the previous histograms

#Delete from the dictionary all_ranks values with key > N:
all_ranks_N = all_ranks.copy()
for key,val in all_ranks_N.items():
    for i in range(N+1,30):
        del all_ranks_N[key][i] 

n_importance_N = {} #Key:feature, Value:n
for key,val in all_ranks_N.items():
    n_importance_N[key] = sum(list(val.values()))

subsets = {} # Key:i, Value:[features]
for i in range(30,-1,-1):
    subset = {}
    for feature,n in n_importance_N.items():
        if n >= i:
            subset[feature] = n
    features = list(subset.keys())
    subsets[i] = features   

#Delete duplicates from the previous dictionary:
temp = []
subsets_unique = dict()

for key, val in subsets.items():
    if val not in temp:
        temp.append(val)
        subsets_unique[key] = val

for key, val in subsets_unique.items():        
    #Add the feature subset to Candidemia, BDG and PCT in both sets:
    sub2_BDG_PCT_y_training = training_BDG_PCT_std[['CANDIDEMIA','B_D_GLUCANO','PROCALCITONINA']+val]
    sub2_BDG_PCT_y_test = test_BDG_PCT_std[['CANDIDEMIA','B_D_GLUCANO','PROCALCITONINA']+val]

    #Splitting features and outcome:
    X_training = sub2_BDG_PCT_y_training.iloc[:,1:sub2_BDG_PCT_y_training.shape[1]].values
    y_training = sub2_BDG_PCT_y_training.iloc[:,0].values
    y_training = np.double(y_training)
    X_test = sub2_BDG_PCT_y_test.iloc[:,1:sub2_BDG_PCT_y_test.shape[1]].values
    y_test = sub2_BDG_PCT_y_test.iloc[:,0].values
    y_test = np.double(y_test)

    #Range of threshold to evaluate:
    thresholds = np.linspace(0.15,0.20,11) 

    #Classifiers to train:
    classifiers = ['RF'] #LogisticRegL1 #LogisticRegL2
    metrics_cl = {}

    for cl in classifiers:
        p = {}
        p['model_name'] = cl
        p = initiate_p(p)
        print('Model name:', p['model_name'])

        metrics_th = {}

        for th in tqdm(thresholds):
            p['threshold'] = th
            print('Threshold:', p['threshold'])

            performance = {}
            performance['features'] = sub2_BDG_PCT_y_training.columns
            #---------------------------------------------
            #Hyperparameters search and model fit on training set:    
            if p['model_name'] == 'LogisticRegL1':
                score = make_scorer(my_tss_score,threshold=th,needs_proba=True)
                fit = LogisticRegressionCV(cv = 10,random_state = 100,penalty='l1',solver='liblinear',scoring=score).fit(X_training, y_training) 

            elif p['model_name'] == 'LogisticRegL2':
                score = make_scorer(my_tss_score,threshold=th,needs_proba=True)
                fit = LogisticRegressionCV(cv = 10,random_state = 100,penalty='l2',scoring=score).fit(X_training, y_training) 

            elif p['model_name'] == 'RF':
                p['grid'] = build_grid(p)
                p['GS_RF'] = GridSearch_(X_training, y_training, p)
                max_depth = p['GS_RF'].best_params_['max_depth']
                n_estimators = p['GS_RF'].best_params_['n_estimators']
                criterion = p['GS_RF'].best_params_['criterion']
                max_features = p['GS_RF'].best_params_['max_features']
                print(p['GS_RF'].best_params_)
                performance['best_hyper'] = p['GS_RF'].best_params_
                fit = RandomForestClassifier(max_depth=max_depth,n_estimators=n_estimators,
                                             criterion=criterion,max_features=max_features,random_state=100).fit(X_training, y_training) 

            #other models can be added here
            
            #---------------------------------------------
            #Predict on training set:
            PRED_prob_tr = fit.predict_proba(X_training)
            PRED_tr_yes = PRED_prob_tr[:,1] 

            PRED_tr_bin = PRED_tr_yes > th
            PRED_tr_bin = PRED_tr_bin*1.

            performance['ytr'] = y_training
            performance['PRED_prob_tr'] = PRED_prob_tr

            #Compute evaluation metrics on training set:
            print(confusion_matrix(y_training,PRED_tr_bin))
            tn_tr, fp_tr, fn_tr, tp_tr = confusion_matrix(y_training, PRED_tr_bin).ravel()
            spec_tr = tn_tr / (tn_tr+fp_tr)
            f1_tr = f1_score(y_training, PRED_tr_bin,average = 'weighted')
            acc_tr = accuracy_score(y_training, PRED_tr_bin)
            prec_tr = precision_score(y_training, PRED_tr_bin,average = 'weighted')
            recall_tr = recall_score(y_training, PRED_tr_bin) 
            npv_tr = tn_tr / (tn_tr+fn_tr)

            performance['tss_tr'] = recall_tr+spec_tr-1
            performance['f1score_tr'] = f1_tr
            performance['accuracy_tr'] = acc_tr
            performance['precision_tr'] = prec_tr
            performance['recall_tr'] = recall_tr
            performance['specificity_tr'] = spec_tr 
            performance['npv_tr'] = npv_tr 

            #---------------------------------------------
            #Predict on test set:
            PRED_prob_ts = fit.predict_proba(X_test)
            PRED_ts_yes = PRED_prob_ts[:,1]

            PRED_ts_bin = PRED_ts_yes>th
            PRED_ts_bin = PRED_ts_bin*1.

            performance['yts'] = y_test
            performance['PRED_prob_ts'] = PRED_prob_ts

            #Compute evaluation metrics on test set:
            print(confusion_matrix(y_test,PRED_ts_bin))
            tn_ts, fp_ts, fn_ts, tp_ts = confusion_matrix(y_test,PRED_ts_bin).ravel()
            spec_ts = tn_ts / (tn_ts+fp_ts)
            f1_ts = f1_score(y_test, PRED_ts_bin,average = 'weighted')
            acc_ts = accuracy_score(y_test, PRED_ts_bin)
            prec_ts = precision_score(y_test, PRED_ts_bin,average = 'weighted')
            recall_ts = recall_score(y_test, PRED_ts_bin)
            npv_ts = tn_ts / (tn_ts+fn_ts)

            performance['tss_ts'] = recall_ts+spec_ts-1
            performance['f1score_ts'] = f1_ts
            performance['accuracy_ts'] = acc_ts
            performance['precision_ts'] = prec_ts
            performance['recall_ts'] = recall_ts
            performance['specificity_ts'] = spec_ts
            performance['npv_ts'] = npv_ts 
            #---------------------------------------------
            metrics_th[th] = performance
        #---------------------------------------------
        #Performances at each treshold:
        metrics_cl[cl] = metrics_th
        #Save on file:
        #nome_file = '<insert your path>/Zoom/Cutoff' + str(N) + '_' + cl + '_Performances_rank'+ str(key)+'.npy' 
        #np.save(nome_file, metrics_th)
    #Save on file:
    #nome_file = '<insert your path>/Zoom/Complete_Performances_Cutoff_' + str(N)+'.npy'
    #np.save(nome_file, metrics_cl)

In [ ]:
#insert your path here:
all_ranks = np.load('<insert your path>/all_ranks.npy', allow_pickle = True).item()

In [ ]:
#Performances visualization (TSS, accuracy, F1-score, precision, NPV) on training set (Figure S5):
N = [30,23,22,21,20,19,18,16,15,13,12,10,9,8,6,0] #key of subset_unique
thresholds = np.linspace(0.15,0.20,11)

for th in thresholds:
    n = [] #number of features plus BDG and PCT
    tss = []
    f1 = []
    acc = []
    prec = []
    se = []
    sp = []
    npv = []
    
    for i in N:
        #insert your path here:
        nome_file = '<insert your path>/Zoom/Cutoff18_RF_Performances_rank' + str(i) +'.npy'
        file = np.load(nome_file, allow_pickle = True).item()
        metrics = file[th]
        
        n.append(len(metrics['features'])-3)
        tss.append(metrics['tss_tr'])
        f1.append(metrics['f1score_tr'])
        acc.append(metrics['accuracy_tr'])
        prec.append(metrics['precision_tr'])
        npv.append(metrics['npv_tr'])
        se.append(metrics['recall_tr'])
        sp.append(metrics['specificity_tr'])
        
        #Likelihood Ratios:
        #pos_LR_tr, neg_LR_tr = class_likelihood_ratios(metrics['ytr'],PRED_tr_bin)
        #lr_p.append(pos_LR_tr)
        #lr_n.append(neg_LR_tr)
        
    print(tss)
        
    #performances to visualize:
    y = np.array([tss,f1,acc,prec,npv]).transpose() #,lr_p,lr_n
    plt.xlabel("N")
    plt.ylabel("Model performance")
    plt.ylim(-0.1, 1.1)
    plt.title(str(th))

    legend = ['TSS', 'F1-score', 'Accuracy', 'Precision','NPV'] #, "LR+","LR-"
    colors = ['#fdb462','#b3de69','#bebada','#fb8072','#80b1d3'] #,'#8dd3c7','#ffffb3'

    for i in range(len(y[0])):
        plt.plot(n,[pt[i] for pt in y],label = legend[i], color = colors[i])
    plt.legend(loc=(0.82,0.22),fontsize='x-small')
    plt.xticks(n,size=6.5)
    plt.yticks(size=6.5)
    #nome_figura = '<insert your path>/Zoom/Model_tss_f1_acc_prec_trend_npv_' + str(th) +'.png'
    #plt.savefig(nome_figura,dpi=300)
    plt.show()

In [ ]:
#Performances visualization (recall, SP) on training set (Figure S5):
N = [30,23,22,21,20,19,18,16,15,13,12,10,9,8,6,0] #key of subset_unique
thresholds = np.linspace(0.15,0.20,11)

for th in thresholds:
    n = [] #number of features plus BDG and PCT
    tss = []
    f1 = []
    acc = []
    prec = []
    se = []
    sp = []
    
    for i in N:
        #insert your path here:
        nome_file = '<insert your path>/Zoom/Cutoff18_RF_Performances_rank' + str(i) +'.npy'
        file = np.load(nome_file, allow_pickle = True).item()
        metrics = file[th]
        
        n.append(len(metrics['features'])-3)
        tss.append(metrics['tss_tr'])
        f1.append(metrics['f1score_tr'])
        acc.append(metrics['accuracy_tr'])
        prec.append(metrics['precision_tr'])
        npv.append(metrics['npv_tr'])
        se.append(metrics['recall_tr'])
        sp.append(metrics['specificity_tr'])
        
        #Likelihood Ratios:
        #pos_LR_tr, neg_LR_tr = class_likelihood_ratios(metrics['ytr'],PRED_tr_bin)
        #lr_p.append(pos_LR_tr)
        #lr_n.append(neg_LR_tr)
        
    #performances to visualize:
    y = np.array([se,sp]).transpose() 
    plt.xlabel("N")
    plt.ylabel("Model performance")
    plt.ylim(-0.1, 1.1)
    #plt.title(str(th))

    legend = ['Recall', 'Specificity']
    colors = ['#8dd3c7','#fff033']

    for i in range(len(y[0])):
        plt.plot(n,[pt[i] for pt in y],label = legend[i], color = colors[i])
    plt.legend(loc=(0.82,0.22),fontsize='x-small')
    plt.xticks(n,size=6.5)
    plt.yticks(size=6.5)
    plt.axhline(y=0.6, color='black', linestyle='-.',linewidth=0.7)
    #nome_figura = '<insert your path>/Zoom/Model_se_sp_trend_' + str(th) +'.png'
    #plt.savefig(nome_figura,dpi=300)
    plt.show()

In [ ]:
#Comparison of performances' mean and standard deviation for subsets with 9<=N<=26, given each threshold between 0.15 and 0.18 (Table S5):
N = [30,23,22,21,20,19,18,16,15,13,12,10,9,8,6,0] #key of subset_unique
thresholds = np.linspace(0.15,0.20,11)

new_thresholds_training = {}
new_thresholds_test = {}

for th in thresholds:
    if th >=0.15 and th <=0.18:
        
        new_thresholds_training[th] = {}
        new_thresholds_test[th] = {}
        
        tss_tr,f1_tr,accuracy_tr,precision_tr,recall_tr,specificity_tr,npv_tr,lr_pos_tr,lr_neg_tr = [],[],[],[],[],[],[],[],[]
        tss_ts,f1_ts,accuracy_ts,precision_ts,recall_ts,specificity_ts,npv_ts,lr_pos_ts,lr_neg_ts = [],[],[],[],[],[],[],[],[]
        
        for i in N:
            #isert your path here:
            nome_file = '<insert your path>/Zoom/Cutoff18_RF_Performances_rank' + str(i) +'.npy'
            file = np.load(nome_file, allow_pickle = True).item()
            metrics = file[th]
            num_feat = len(metrics['features'])-3
            

            if num_feat >= 9 and num_feat <= 26:
                tss_tr.append(metrics['tss_tr'])
                f1_tr.append(metrics['f1score_tr'])
                accuracy_tr.append(metrics['accuracy_tr'])
                precision_tr.append(metrics['precision_tr'])
                npv_tr.append(metrics['npv_tr'])
                recall_tr.append(metrics['recall_tr'])
                specificity_tr.append(metrics['specificity_tr'])
                
                pos_LR_tr, neg_LR_tr = class_likelihood_ratios(metrics['ytr'],PRED_tr_bin)
                lr_pos_tr.append(pos_LR_tr)
                lr_neg_tr.append(neg_LR_tr)
                
                tss_ts.append(metrics['tss_ts'])
                f1_ts.append(metrics['f1score_ts'])
                accuracy_ts.append(metrics['accuracy_ts'])
                precision_ts.append(metrics['precision_ts'])
                npv_ts.append(metrics['npv_ts'])
                recall_ts.append(metrics['recall_ts'])
                specificity_ts.append(metrics['specificity_ts'])
                
                pos_LR_ts, neg_LR_ts = class_likelihood_ratios(metrics['yts'],PRED_ts_bin)
                lr_pos_ts.append(pos_LR_ts)
                lr_neg_ts.append(neg_LR_ts)
                
            new_thresholds_training[th]['TSS'] = tss_tr
            new_thresholds_training[th]['F1'] = f1_tr
            new_thresholds_training[th]['ACC'] = accuracy_tr
            new_thresholds_training[th]['PREC'] = precision_tr
            new_thresholds_training[th]['SE'] = recall_tr
            new_thresholds_training[th]['SP'] = specificity_tr
            new_thresholds_training[th]['NPV'] = npv_tr
            new_thresholds_training[th]['LR+'] = lr_pos_tr
            new_thresholds_training[th]['LR-'] = lr_neg_tr
            
            
            new_thresholds_test[th]['TSS'] = tss_ts
            new_thresholds_test[th]['F1'] = f1_ts
            new_thresholds_test[th]['ACC'] = accuracy_ts
            new_thresholds_test[th]['PREC'] = precision_ts
            new_thresholds_test[th]['SE'] = recall_ts
            new_thresholds_test[th]['SP'] = specificity_ts
            new_thresholds_test[th]['NPV'] = npv_ts
            new_thresholds_test[th]['LR+'] = lr_pos_ts
            new_thresholds_test[th]['LR-'] = lr_neg_ts

print('On training set:')
for k in list(new_thresholds_training.keys()):
    print('Threshold' ,k)
    print('TSS mean value and standard deviation are: ', np.mean(new_thresholds_training[k]['TSS']),',' ,np.std(new_thresholds_training[k]['TSS']))
    print('F1-score mean value and standard deviation are: ', np.mean(new_thresholds_training[k]['F1']),',' ,np.std(new_thresholds_training[k]['F1']))
    print('Accuracy mean value and standard deviation are: ', np.mean(new_thresholds_training[k]['ACC']),',' ,np.std(new_thresholds_training[k]['ACC']))
    print('Precision mean value and standard deviation are: ', np.mean(new_thresholds_training[k]['PREC']),',' ,np.std(new_thresholds_training[k]['PREC']))
    print('Recall mean value and standard deviation are: ', np.mean(new_thresholds_training[k]['SE']),',' ,np.std(new_thresholds_training[k]['SE']))
    print('Specificity mean value and standard deviation are: ', np.mean(new_thresholds_training[k]['SP']),',' ,np.std(new_thresholds_training[k]['SP']))
    print('NPV mean value and standard deviation are: ', np.mean(new_thresholds_training[k]['NPV']),',' ,np.std(new_thresholds_training[k]['NPV']))
    print('LR+ mean value and standard deviation are: ', np.mean(new_thresholds_training[k]['LR+']),',' ,np.std(new_thresholds_training[k]['LR+']))
    print('LR- mean value and standard deviation are: ', np.mean(new_thresholds_training[k]['LR-']),',' ,np.std(new_thresholds_training[k]['LR-']))
print('\n')   
print('On test set:')
for k in list(new_thresholds_test.keys()):
    print('Threshold' ,k)
    print('TSS mean value and standard deviation are: ', np.mean(new_thresholds_test[k]['TSS']),',' ,np.std(new_thresholds_test[k]['TSS']))
    print('F1-score mean value and standard deviation are: ', np.mean(new_thresholds_test[k]['F1']),',' ,np.std(new_thresholds_test[k]['F1']))
    print('Accuracy mean value and standard deviation are: ', np.mean(new_thresholds_test[k]['ACC']),',' ,np.std(new_thresholds_test[k]['ACC']))
    print('Precision mean value and standard deviation are: ', np.mean(new_thresholds_test[k]['PREC']),',' ,np.std(new_thresholds_test[k]['PREC']))
    print('Recall mean value and standard deviation are: ', np.mean(new_thresholds_test[k]['SE']),',' ,np.std(new_thresholds_test[k]['SE']))
    print('Specificity mean value and standard deviation are: ', np.mean(new_thresholds_test[k]['SP']),',' ,np.std(new_thresholds_test[k]['SP']))
    print('NPV mean value and standard deviation are: ', np.mean(new_thresholds_test[k]['NPV']),',' ,np.std(new_thresholds_test[k]['NPV']))
    print('LR+ mean value and standard deviation are: ', np.mean(new_thresholds_test[k]['LR+']),',' ,np.std(new_thresholds_test[k]['LR+']))
    print('LR- mean value and standard deviation are: ', np.mean(new_thresholds_test[k]['LR-']),',' ,np.std(new_thresholds_test[k]['LR-']))

###### <font color='indianred'><b>CHOICE 1: Threshold = 0.175</b></font>

###### <font color='indianred'><b>CHOICE 2: Subset with 12 features plus BDG and PCT</b></font>

In [ ]:
#No other data set needs to be exported for following analysis